<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Advent-of-code-2019" data-toc-modified-id="Advent-of-code-2019-1">Advent of code 2019</a></span><ul class="toc-item"><li><span><a href="#Day-10" data-toc-modified-id="Day-10-1.1">Day 10</a></span><ul class="toc-item"><li><span><a href="#Part-1" data-toc-modified-id="Part-1-1.1.1">Part 1</a></span></li><li><span><a href="#Part-2" data-toc-modified-id="Part-2-1.1.2">Part 2</a></span></li></ul></li></ul></li></ul></div>

# Advent of code 2019

https://adventofcode.com/2019

## Day 10

### Part 1

In [1]:
def test_max_in_sight(f):
    assert f((
"""
.#..#
.....
#####
....#
...##
""").strip('\n')) == 8
    assert f((
"""
......#.#.
#..#.#....
..#######.
.#.#.###..
.#..#.....
..#....#.#
#..#....#.
.##.#..###
##...#..#.
.#....####
""").strip('\n')) == 33
    assert f((
"""
#.#...#.#.
.###....#.
.#....#...
##.#.#.#.#
....#.#.#.
.##..###.#
..#...##..
..##....##
......#...
.####.###.
""").strip('\n')) == 35
    assert f((
"""
.#..#..###
####.###.#
....###.#.
..###.##.#
##.##.#.#.
....###..#
..#.#..#.#
#..#.#.###
.##...##.#
.....#.#..
""").strip('\n')) == 41
    assert f((
"""
.#..##.###...#######
##.############..##.
.#.######.########.#
.###.#######.####.#.
#####.##.#.##.###.##
..#####..#.#########
####################
#.####....###.#.#.##
##.#################
#####.##.###..####..
..######..##.#######
####.##.####...##..#
.#####..#.######.###
##...#.##########...
#.##########.#######
.####.#.###.###.#.##
....##.##.###..#####
.#.#.###########.###
#.#.#.#####.####.###
###.##.####.##.#..##
""").strip('\n')) == 210

In [2]:
def get_asteroid_pts(asteroid_map):
    asteroid_pts = []
    for i, row in enumerate(asteroid_map.split('\n')):
        asteroid_pts.extend([(j, i) for j, u in enumerate(row) if u == '#'])
    return asteroid_pts    

In [3]:
def calc_vector(pt1, pt2):
    return pt2[0] - pt1[0], pt2[1] - pt1[1]

from numpy import gcd
def reduce_vector(a, b):
    d = gcd(a, b)
    return a//d, b//d

def calc_vectors_in_sight(asteroid_pts):
    N = len(asteroid_pts)

    vectors = [set() for _ in range(N)]
    for i in range(N):
        for j in range(i+1, N):
            a, b = calc_vector(asteroid_pts[i], asteroid_pts[j])
            a, b = reduce_vector(a, b)
            vectors[i].add((a, b))
            vectors[j].add((-a, -b))
    
    return vectors

In [4]:
def get_max_in_sight(asteroid_pts):
    vectors = calc_vectors_in_sight(asteroid_pts)
    
    num_in_sight = [len(u) for u in vectors]
    max_num_in_sight = max(num_in_sight)
    # print(asteroid_pts[num_in_sight.index(max_num_in_sight)])
    
    return max_num_in_sight

def get_max_in_sight_from_map(asteroid_map):
    return get_max_in_sight(get_asteroid_pts(asteroid_map))

In [5]:
test_max_in_sight(get_max_in_sight_from_map)

In [7]:
input_file = 'input/input_day10.txt'
with open(input_file, 'r') as fin:
    data_map = fin.read()

In [8]:
get_max_in_sight_from_map(data_map)

347

### Part 2

In [9]:
def test_200_asteroid(f):
    assert f((
"""
.#..##.###...#######
##.############..##.
.#.######.########.#
.###.#######.####.#.
#####.##.#.##.###.##
..#####..#.#########
####################
#.####....###.#.#.##
##.#################
#####.##.###..####..
..######..##.#######
####.##.####...##..#
.#####..#.######.###
##...#.##########...
#.##########.#######
.####.#.###.###.#.##
....##.##.###..#####
.#.#.###########.###
#.#.#.#####.####.###
###.##.####.##.#..##
""").strip('\n')) == (8, 2)

In [10]:
def get_best_asteroid(asteroid_pts):
    vectors = calc_vectors_in_sight(asteroid_pts)
    
    num_in_sight = [len(u) for u in vectors]
    max_num_in_sight = max(num_in_sight)
    return asteroid_pts[num_in_sight.index(max_num_in_sight)]

def get_relative_vectors(asteroid_pts):
    best_pt = get_best_asteroid(asteroid_pts)
    return [(pos[0] - best_pt[0], -pos[1] + best_pt[1]) for pos in asteroid_pts if pos != best_pt]

In [11]:
from numpy import arctan2, degrees, abs

def get_angle(vector):
    return (90 - degrees(arctan2(vector[1], vector[0]))) % 360

# def get_angles(vectors):
#     return [get_angle(vec) for vec in vectors]

def shift_angle(angle, shift):
    return (angle + shift) % 360

def mag_vector(vector):
    return abs(vector[0]) + abs(vector[1])

In [12]:
def eliminate_asteroid(asteroid_pts):
    best_pt = get_best_asteroid(asteroid_pts)
    relative_vectors = get_relative_vectors(asteroid_pts)
    
    dict_angles = {u: get_angle(u) for u in relative_vectors}
    dict_mag = {u: mag_vector(u) for u in relative_vectors}
    
    while dict_angles:
        min_angle = min(dict_angles.values())
        asteroids_in_sight = [pt for pt, v in dict_angles.items() if v == min_angle]
        dists = [dict_mag[u] for u in asteroids_in_sight]

        min_dist = min(dists)
        asteroid_to_remove = asteroids_in_sight[dists.index(min_dist)]

        dict_angles.pop(asteroid_to_remove)

        dict_angles = {u: shift_angle(v, -min_angle-0.001) for u, v in dict_angles.items()}
        
        # print(asteroid_to_remove, get_angle(asteroid_to_remove))
        yield (best_pt[0] + asteroid_to_remove[0], best_pt[1] - asteroid_to_remove[1])

In [13]:
def clear_asteroid(asteroid_map, num=200):
    asteroid_pts = get_asteroid_pts(asteroid_map)

    for i, val in enumerate(eliminate_asteroid(asteroid_pts)):
        if i == num-1:
            return val

    return None

In [14]:
test_200_asteroid(clear_asteroid)

In [15]:
clear_asteroid(data_map)

(8, 29)